In [1]:
!python -m spacy download en_core_web_lg

2023-08-04 06:36:08.481661: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-04 06:36:10.355102: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

Politics: 0 Technology: 1 Entertainment: 2 Business: 3

In [9]:
import pandas as pd
train = pd.read_csv("Data_Train.csv", encoding='ISO-8859-1')
train.head()

,STORY,SECTION
0,But the most painful was the huge reversal in ...,3
1,How formidable is the opposition alliance amon...,0
2,Most Asian currencies were trading lower today...,3
3,"If you want to answer any question, click on ...",1
4,"In global markets, gold prices edged up today ...",3


In [11]:
train.SECTION.value_counts()

1    2772
2    1924
0    1686
3    1246
Name: SECTION, dtype: int64

In [12]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [13]:
train['prepro'] = train['STORY'].apply(preprocess)
train.head()

,STORY,SECTION,prepro
0,But the most painful was the huge reversal in ...,3,painful huge reversal fee income unheard priva...
1,How formidable is the opposition alliance amon...,0,formidable opposition alliance Congress Jharkh...
2,Most Asian currencies were trading lower today...,3,asian currency trade low today south korean wi...
3,"If you want to answer any question, click on ...",1,want answer question click answer. click ans...
4,"In global markets, gold prices edged up today ...",3,global market gold price edge today disappoint...


In [15]:
train['vector'] = train['prepro'].apply(lambda x: nlp(x).vector)
train.head()

,STORY,SECTION,prepro,vector
0,But the most painful was the huge reversal in ...,3,painful huge reversal fee income unheard priva...,"[0.31490126, 0.23004095, -2.1700723, -0.725907..."
1,How formidable is the opposition alliance amon...,0,formidable opposition alliance Congress Jharkh...,"[-0.7317262, 0.6593279, 0.027785195, 0.3549192..."
2,Most Asian currencies were trading lower today...,3,asian currency trade low today south korean wi...,"[-0.700607, -0.4616842, -0.6389873, 0.92671204..."
3,"If you want to answer any question, click on ...",1,want answer question click answer. click ans...,"[2.2761981, 1.2820551, 0.9615063, 0.13240609, ..."
4,"In global markets, gold prices edged up today ...",3,global market gold price edge today disappoint...,"[0.067549296, 0.14487036, -2.8356595, 2.050999..."


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    train.vector.values,
    train.SECTION,
    test_size=0.2,
    random_state=2022
)

In [24]:
X_train[0]

array([ 1.7737042e-02,  7.9581207e-01, -5.1668203e-01,  9.4453108e-01,
        2.8559155e+00,  1.3488436e-01,  8.0259770e-01,  1.7666465e+00,
       -8.1037813e-01, -4.0107700e-01,  2.2413249e+00,  9.0910459e-01,
       -2.5543203e+00,  1.1646359e+00, -2.4028240e-01,  1.0610082e+00,
        2.3130322e+00, -4.7921643e-02,  3.3525643e-01,  7.9052508e-02,
        1.2068301e+00,  1.0938010e+00, -1.4483131e-01,  9.0064073e-01,
       -2.1323035e+00, -2.9634643e-01, -3.3699529e+00, -1.6120172e+00,
        3.6405692e-01,  9.6142083e-01,  2.0148790e+00,  3.7920293e-01,
       -1.6377993e-01, -1.6245786e+00,  4.8671207e-01, -5.8259195e-01,
       -7.7233410e-01,  1.4960409e+00,  9.2198688e-01, -1.1644790e-01,
        9.2423819e-02,  6.8542069e-01, -9.4495422e-01,  6.1690480e-01,
       -6.0893190e-01,  2.3598723e-02,  5.9073544e-01,  2.9522637e-01,
        1.2043936e+00, -2.3266545e-02,  5.3294641e-01,  7.8462285e-01,
       -9.1348135e-01, -1.7459964e+00, -1.5106983e-01,  1.0896059e+00,
      

In [17]:
import numpy as np
X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [18]:
X_train_2d

array([[ 0.01773704,  0.7958121 , -0.516682  , ..., -2.138774  ,
        -0.49456337,  0.8228577 ],
       [ 0.69585055,  1.6822011 , -0.32800975, ..., -1.0918703 ,
        -1.4364353 , -0.2078367 ],
       [-0.60755885,  0.4742088 , -1.0523454 , ..., -1.5877546 ,
        -0.7792396 ,  1.4265304 ],
       ...,
       [-2.4805675 ,  0.16298598,  0.82185656, ...,  1.0908439 ,
        -0.980512  ,  0.7503535 ],
       [-0.9530591 ,  0.70670813, -0.5934083 , ..., -1.6365162 ,
        -1.0259886 ,  1.5130165 ],
       [-0.74217004,  0.7938517 , -0.9041328 , ..., -1.4781034 ,
        -1.0832504 ,  1.0135375 ]], dtype=float32)

In [22]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

dtc = DecisionTreeClassifier()

dtc.fit(X_train_2d, y_train)

y_pred = dtc.predict(X_test_2d)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.84      0.84       339
           1       0.85      0.86      0.85       550
           2       0.86      0.82      0.84       394
           3       0.77      0.81      0.79       243

    accuracy                           0.84      1526
   macro avg       0.83      0.83      0.83      1526
weighted avg       0.84      0.84      0.84      1526



In [25]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

mn = MultinomialNB()
mms = MinMaxScaler()

scaled_train = mms.fit_transform(X_train_2d)
scaled_test = mms.transform(X_test_2d)

mn.fit(scaled_train, y_train)

y_pred = mn.predict(scaled_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89       339
           1       0.87      0.93      0.90       550
           2       0.93      0.89      0.91       394
           3       0.90      0.80      0.85       243

    accuracy                           0.89      1526
   macro avg       0.90      0.88      0.89      1526
weighted avg       0.89      0.89      0.89      1526



In [26]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

knn.fit(scaled_train, y_train)

y_pred = knn.predict(scaled_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.97      0.92       339
           1       0.94      0.96      0.95       550
           2       0.97      0.89      0.93       394
           3       0.96      0.93      0.95       243

    accuracy                           0.94      1526
   macro avg       0.94      0.94      0.94      1526
weighted avg       0.94      0.94      0.94      1526



In [27]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(scaled_train, y_train)

y_pred = rfc.predict(scaled_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.93      0.94       339
           1       0.92      0.93      0.93       550
           2       0.95      0.93      0.94       394
           3       0.90      0.93      0.91       243

    accuracy                           0.93      1526
   macro avg       0.93      0.93      0.93      1526
weighted avg       0.93      0.93      0.93      1526



In [29]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()

gbc.fit(scaled_train, y_train)

y_pred = gbc.predict(scaled_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       339
           1       0.95      0.93      0.94       550
           2       0.95      0.94      0.95       394
           3       0.92      0.95      0.93       243

    accuracy                           0.94      1526
   macro avg       0.94      0.94      0.94      1526
weighted avg       0.94      0.94      0.94      1526

